In [1]:
import os
os.environ["CC"] = "/usr/bin/gcc-11"
os.environ["CXX"] = "/usr/bin/g++-11"
import struct
import numpy as np
from tqdm import tqdm

from pathlib import Path
import imageio
import os
import sys
sys.path.append(str(Path(os.path.abspath('')).parent))
print(str(Path(os.path.abspath('')).parent))
import torch
from gaussian_renderer import GaussianModel, splinerender, render
from scene import Scene
from scene.cameras import Camera, MiniCam
from torch import nn
import matplotlib.pyplot as plt
import imageio
from pyquaternion import Quaternion
from scene.dataset_readers import ProjectionType

dataset = "berlin"
eval_path = "/home/amai/gaussian-splatting-merge/eval.znf"
# eval_path = str(Path("/cold/gaussian-splatting-merge.bk2/eval.znf"))
output_path = Path(f"/home/amai/Videos/rotating_rooms/render")

output_path.mkdir(parents=True, exist_ok=True)
def load_path(path):
    f = open(path, "rb")
    data = f.read()
    N = int.from_bytes(data[:4])
    camera_size = 11

    cameras = np.array(struct.unpack(f'>{N*camera_size}f', data[4:])).reshape(N, -1)
    full_data = struct.unpack(f'>i{N*camera_size}f', data)
    return cameras

/home/amai/gaussian-splatting-merge
['-std=c++17']


In [2]:
plane_points = load_path("/home/amai/Videos/rotating_rooms/cutting_plane.path")[:, :3]
point_on_plane = plane_points[0:1]
normed = plane_points-point_on_plane
evals, evecs = np.linalg.eig(normed.T @ normed)
normal = np.cross(evecs[0], evecs[1])
print(normal)

[ 0.00167627 -0.12787477 -0.99178891]


In [23]:
cameras = load_path("/home/amai/Videos/rotating_rooms/video_path.path")#[400:]

In [4]:
import shlex
from argparse import ArgumentParser, Namespace
from arguments import ModelParams, PipelineParams, OptimizationParams

def get_combined_args(args_cmdline):
    cfgfile_string = "Namespace()"

    try:
        cfgfilepath = os.path.join(args_cmdline.model_path, "cfg_args")
        print("Looking for config file in", cfgfilepath)
        with open(cfgfilepath) as cfg_file:
            print("Config file found: {}".format(cfgfilepath))
            cfgfile_string = cfg_file.read()
    except TypeError:
        print("Config file not found at")
        pass
    args_cfgfile = eval(cfgfile_string)

    merged_dict = vars(args_cfgfile).copy()
    for k, v in vars(args_cmdline).items():
        if v != None:
            merged_dict[k] = v
    return Namespace(**merged_dict)


parser = ArgumentParser(description="Testing script parameters")
model = ModelParams(parser, sentinel=True)
pipeline = PipelineParams(parser)
args = parser.parse_args(shlex.split(f"-m {Path(eval_path) / dataset} --images images_4 -r 1"))
print(args.model_path)
args = get_combined_args(args)
model = model.extract(args)
# model.source_path = str(Path("/data/nerf_synthetic") / dataset)
# model.source_path = str(Path("/data/nerf_datasets/tandt/") / dataset)
model.source_path = str(Path("/data/nerf_datasets/zipnerf") / dataset)

model.max_opacity = 0.99

pipeline = pipeline.extract(args)

/home/amai/gaussian-splatting-merge/eval.znf/berlin
Looking for config file in /home/amai/gaussian-splatting-merge/eval.znf/berlin/cfg_args
Config file found: /home/amai/gaussian-splatting-merge/eval.znf/berlin/cfg_args


In [5]:
gaussians = GaussianModel(model.sh_degree, model.max_opacity)
scene = Scene(model, gaussians, load_iteration=-1, shuffle=False)

ic| self.max_opacity: 0.99


Loading trained model at iteration 30000
Reading camera 1525/1525
Loading Training Cameras
Loaded Train Cameras: 1525
Loaded Test Cameras: 0


In [6]:
refcam = scene.getTrainCameras()[0]

In [7]:
from gaussian_renderer.fast_renderer import FastRenderer

renderer = FastRenderer(refcam, gaussians, False)


In [8]:
original_locations = gaussians._xyz.data.clone()
original_rots = gaussians.get_rotation.data.clone()

In [14]:
axis_points = load_path("/home/amai/Videos/rotating_rooms/ceiling_plane.path")[:, :3]
origin = axis_points[0:1]
direction = axis_points[-1:] - origin
direction = (direction / np.linalg.norm(direction)).reshape(1, -1)
origin = torch.as_tensor(origin).float().cuda()
direction = torch.as_tensor(direction).float().cuda()

ceiling_mask = (direction @ (original_locations - origin).T > 0).reshape(-1)

axis_points = load_path("/home/amai/Videos/rotating_rooms/floor_plane.path")[:, :3]
origin = axis_points[0:1]
direction = axis_points[-1:] - origin
direction = (direction / np.linalg.norm(direction)).reshape(1, -1)
origin = torch.as_tensor(origin).float().cuda()
direction = torch.as_tensor(direction).float().cuda()

floor_mask = (direction @ (original_locations - origin).T > 0).reshape(-1)


In [15]:
axis_points = load_path("/home/amai/Videos/rotating_rooms/rotation_axis.path")[:, :3]
origin = axis_points[0:1]
direction = axis_points[-1:] - origin
direction = (direction / np.linalg.norm(direction)).reshape(1, -1)
origin = torch.as_tensor(origin).float().cuda()
direction = torch.as_tensor(direction).float().cuda()

mask = (direction @ (original_locations - origin).T > 0).reshape(-1)
mask_offset = (direction @ (original_locations - origin + direction * 0.2).T > 0).reshape(-1)

In [20]:
axis_points = load_path("/home/amai/Videos/rotating_rooms/rotation_axis2.path")[:, :3]
cut_points = load_path("/home/amai/Videos/rotating_rooms/cutting_plane2.path")[:, :3]
cut_points3 = load_path("/home/amai/Videos/rotating_rooms/cutting_plane3.path")[:, :3]

origin1 = axis_points[0:1]
direction1 = axis_points[-1:] - origin1
direction1 = (direction1 / np.linalg.norm(direction1)).reshape(1, -1)
origin1 = torch.as_tensor(origin1).float().cuda()
direction1 = torch.as_tensor(direction1).float().cuda()

origin2 = cut_points[0:1]
direction2 = cut_points[-1:] - origin2
direction2 = (direction2 / np.linalg.norm(direction2)).reshape(1, -1)
origin2 = torch.as_tensor(origin2).float().cuda()
direction2 = torch.as_tensor(direction2).float().cuda()

origin3 = cut_points3[0:1]
direction3 = cut_points3[-1:] - origin3
direction3 = (direction3 / np.linalg.norm(direction3)).reshape(1, -1)
origin3 = torch.as_tensor(origin3).float().cuda()
direction3 = torch.as_tensor(direction3).float().cuda()

half_plane1 = (direction1 @ (original_locations - origin1).T > 0).reshape(-1)
half_plane2 = (direction2 @ (original_locations - origin2).T > 0).reshape(-1)
half_plane3 = (direction3 @ (original_locations - origin3).T > 0).reshape(-1)

mask2 = half_plane1 & half_plane2 & half_plane3

mask = mask# & ceiling_mask & floor_mask
mask2 = mask2 & ceiling_mask# & floor_mask


In [21]:
def rotate_quaternions(q, r):
  """
  Rotates quaternions 'q' by another quaternion 'r'.

  Args:
      q: A tensor of quaternions to be rotated, shape (..., 4)
      r: A tensor of quaternions representing the rotation, shape (..., 4)

  Returns:
      A tensor of rotated quaternions, shape (..., 4)
  """

  # Ensure both q and r are normalized
  q = q / torch.norm(q, dim=-1, keepdim=True)
  r = r / torch.norm(r, dim=-1, keepdim=True)

  # Extract quaternion components for easier manipulation
  qw, qx, qy, qz = torch.unbind(q, dim=-1)
  rw, rx, ry, rz = torch.unbind(r, dim=-1)

  # Perform the quaternion multiplication (Hamilton product)
  rotated_qw = rw * qw - rx * qx - ry * qy - rz * qz
  rotated_qx = rw * qx + rx * qw + ry * qz - rz * qy
  rotated_qy = rw * qy - rx * qz + ry * qw + rz * qx
  rotated_qz = rw * qz + rx * qy - ry * qx + rz * qw

  # Stack the rotated components back into a quaternion tensor
  rotated_q = torch.stack([rotated_qw, rotated_qx, rotated_qy, rotated_qz], dim=-1)

  return rotated_q

def transform(points, rots, direction, origin, angle):
    quat = Quaternion(axis=direction.cpu().numpy().reshape(-1), radians=angle)
    quat_t = torch.tensor([quat.w, quat.x, quat.y, quat.z]).float().cuda().reshape(1, -1)
    # quat_t = torch.tensor([1, 0, 0, 0]).float().cuda().reshape(1, -1)
    new_rots = rotate_quaternions(rots, quat_t)
    R = torch.as_tensor(quat.rotation_matrix).float().cuda()
    rotated_pts = R @ (points - origin).T
    return rotated_pts.T + origin, new_rots

In [27]:
background = torch.tensor([0, 0, 0], dtype=torch.float32, device="cuda")
# width = 1280
# height = 720
# width = 1200
# height = 667
width = refcam.image_width
height = refcam.image_height
image = torch.ones((3, height, width), dtype=float)
for i in tqdm(range(cameras.shape[0])):
    start = 160
    end = 260
    start2 = 350
    end2 = 580
    angle = (i-start2) / (end2-start2) * 2 * np.pi if i > start2 and i < end2 else 0
    # angle = 0
    locs, rots = transform(original_locations[mask], original_rots[mask], direction, origin, angle)
    angle2 = -(i-start)/(end-start)*2*np.pi if i > start and i < end else 0
    locs2, rots2 = transform(original_locations[mask2], original_rots[mask2], direction1, origin1, angle2)
    gaussians._xyz.data[mask] = locs
    gaussians._rotation.data[mask] = rots
    gaussians._xyz.data[mask2] = locs2
    gaussians._rotation.data[mask2] = rots2
    # print(locs, original_locations[mask])

    T = cameras[i, :3]
    # xyzw
    quat = cameras[i, 3:7]
    R = Quaternion(x=quat[0], y=quat[1], z=quat[2], w=quat[3]).transformation_matrix
    R[:3, 3] = T
    transf = np.linalg.inv(R).T
    # print(R, transf)
    # transf[1, :] = -transf[1, :]
    # transf[2, :] = -transf[2, :]
    transf[:, 1] = -transf[:, 1]
    transf[:, 2] = -transf[:, 2]

    # R = transf[:3, :3]
    # T = transf[:3, 3]
    fovy = cameras[i, -4]
    fovx = cameras[i, -3]
    fovy = refcam.FoVy
    fovx = refcam.FoVx
    znear = cameras[i, -2]
    zfar = cameras[i, -1]
    # view = Camera(0, R, T, aspect*fovy/180*np.pi, fovy/180*np.pi, image, image, "fake", 0)
    world_view_transform = torch.as_tensor(transf).float()
    full_proj_transform = torch.as_tensor(transf).float()
    # fovx = 1.699109673500061
    # fovx = 1.7087104320526123
    # fovx = 1.399527668952942
    view = MiniCam(width, height, fovy, fovx, znear, zfar, world_view_transform, full_proj_transform)
    view.model = ProjectionType.PERSPECTIVE
    with torch.no_grad():
        rendering = splinerender(view, gaussians, pipeline, background, random=False)["render"]
        renderer.set_camera(view)
        # rendering = renderer.render(view, pipeline, background)
        # rendering = splinerender(cam, gaussians, pipeline, background)["render"]
        byte_rendering = (rendering.permute(1, 2, 0).cpu().numpy()*255).clip(min=0, max=255).astype(np.uint8)
    full_output_path = output_path / f"{i:06d}.png"
    imageio.imwrite(str(full_output_path), byte_rendering)
    # plt.imshow(byte_rendering)
    # plt.show()

100%|██████████| 900/900 [06:16<00:00,  2.39it/s]
